# An Implementation of Flower Pollination Algorithm using Python

Note that in this example, the FPA will be used to find the global minima of Six Hump Camel Function

In [76]:
import numpy as np
from scipy.stats import levy
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import animation

In [24]:
df = pd.read_excel('dataset.xlsx')
df = df.drop('Unnamed: 0', axis=1)
df['periode'] = df['Tahun'].astype(str) + ' ' + df['Bulan']
df = df.drop(['Tahun', 'Bulan'], axis=1)
df = df.set_index('periode')
df.head(10)

,Migas,Non Migas,Jumlah
periode,,,
2020 January,2069.7,48830.4,50900.1
2020 February,2166.9,47504.3,49671.2
2020 March,2007.4,52061.4,54068.8
2020 April,1929.8,43242.9,45172.7
2020 May,2153.5,40082.7,42236.2
2020 June,2288.9,44098.9,46387.8
2020 July,2260.6,43809.3,46069.9
2020 August,2204.9,41360.3,43565.2
2020 September,2478.2,41454.9,43933.1


In [97]:
y = df['Migas']
lookback = 6

def predict(x, weights):
    return np.sum(x.values * weights[1:]) + weights[0]

def objective(weights):
    preds = []

    for i in range(lookback, len(df)):
        x = y[i-lookback:i]
        pred = predict(x, weights)
        preds.append(pred)

    preds = np.array(preds)
    target = y[lookback:].values
    mse = np.mean(np.square(target - preds))
    return mse

In [110]:
np.random.seed(42)

def run_fpa(num_flowers, p, gamma, max_iterations, verbose=True):
    # Initialize population
    flowers = np.random.uniform(-1, 1, (num_flowers, lookback+1)) * 10

    # Determine best solution
    g_star = flowers[0]
    for flower in flowers:
        if objective(flower) < objective(g_star):
            g_star = flower

    # print(flowers.round(2))
    if verbose:
        print('Initial best objective:')
        print(objective(g_star))

    # FPA
    results = []
    for t in range(max_iterations):
        for i in range(num_flowers):
            rand = np.random.uniform(0, 1)

            if rand < p:
                # global pollination
                L = levy.rvs(size=flower.shape[0])
                new_flower = flowers[i] + gamma * L * (g_star - flowers[i])
            else:
                # local
                epsilon = np.random.uniform(0, 1)
                j, k = np.random.choice(num_flowers, 2, replace=False)
                new_flower = flowers[i] + epsilon * (flowers[j] - flowers[k])

            # update if better
            new_objective = objective(new_flower)
            if new_objective < objective(flowers[i]):
                flowers[i] = new_flower

        # find new g_star
        for flower in flowers:
            if objective(flower) < objective(g_star):
                g_star = flower

        best_objective = objective(g_star)
        results.append(best_objective)
        if verbose:
            print(f'Iteration {t+1}: {best_objective:.2f}')

    return g_star, results

In [111]:
max_iterations = 10

list_num_flowers = [50, 100, 200]
list_p = [0.1, 0.5, 0.9]
list_gamma = [0.1, 0.01, 0.001]

results = []

for num_flowers in list_num_flowers:
    for p in list_p:
        for gamma in list_gamma:
            print(f'num_flowers={num_flowers}, p={p}, gamma={gamma}')
            g_star, fpa_result = run_fpa(num_flowers, p, gamma, max_iterations, verbose=False)
            results.append({
                'num_flowers': num_flowers,
                'p': p,
                'gamma': gamma,
                'weight': g_star,
                'mse': objective(g_star)
            })

num_flowers=50, p=0.1, gamma=0.1
num_flowers=50, p=0.1, gamma=0.01
num_flowers=50, p=0.1, gamma=0.001
num_flowers=50, p=0.5, gamma=0.1
num_flowers=50, p=0.5, gamma=0.01
num_flowers=50, p=0.5, gamma=0.001
num_flowers=50, p=0.9, gamma=0.1
num_flowers=50, p=0.9, gamma=0.01
num_flowers=50, p=0.9, gamma=0.001
num_flowers=100, p=0.1, gamma=0.1
num_flowers=100, p=0.1, gamma=0.01
num_flowers=100, p=0.1, gamma=0.001
num_flowers=100, p=0.5, gamma=0.1
num_flowers=100, p=0.5, gamma=0.01
num_flowers=100, p=0.5, gamma=0.001
num_flowers=100, p=0.9, gamma=0.1
num_flowers=100, p=0.9, gamma=0.01
num_flowers=100, p=0.9, gamma=0.001
num_flowers=200, p=0.1, gamma=0.1
num_flowers=200, p=0.1, gamma=0.01
num_flowers=200, p=0.1, gamma=0.001
num_flowers=200, p=0.5, gamma=0.1
num_flowers=200, p=0.5, gamma=0.01
num_flowers=200, p=0.5, gamma=0.001
num_flowers=200, p=0.9, gamma=0.1
num_flowers=200, p=0.9, gamma=0.01
num_flowers=200, p=0.9, gamma=0.001


In [117]:
df_results = pd.DataFrame(results)
df_results = df_results.sort_values('mse')
best_mse = df_results.iloc[0]['mse']
print('Best MSE:', best_mse)

Best MSE: 790030.5456287522


In [118]:
g_star = df_results.iloc[0]['weight']
print('g_star:', g_star)

g_star: [ 6.67992823  1.61893624 -1.84550467  1.32723432 -1.70219465  0.09871973
  1.37160944]


In [129]:
def forecast(x, weights, n):
    preds = []
    for i in range(n):
        pred = predict(x, weights)
        preds.append(pred)
        x = x.shift(-1)
        x.iloc[-1] = pred
    return preds


next_months = 14
forecasted_result = forecast(y[-lookback:], g_star, next_months)
print(f'Prediksi {next_months} bulan ke-depan:')
print('\n> '.join(map(str, forecasted_result)))

Prediksi 14 bulan ke-depan:
1593.0460932983362
> 1747.8926770766748
> 661.4484834932201
> 1038.4988777216993
> 181.33858187576826
> 1932.1773687977607
> 1138.2401925022828
> 4437.32326392667
> 2311.346384490997
> 5588.543819683895
> -1414.5883294658756
> 1600.5673893864441
> -10729.06814991111
> -1808.0489686196047
